In [1]:
import pandas as pd
import pygal

In [35]:
df1 = pd.read_csv('../results/logs.tsv', sep='\t')
df2 = pd.read_csv('../results/logs_200_2.tsv', sep='\t', skiprows=[1,2])
df3 = pd.read_csv('../results/logs_200_3.tsv',sep='\t', skiprows=[1,2])
df4 = pd.read_csv('../results/logs_200_4.tsv',sep='\t', skiprows=[1,2])
df5 = pd.read_csv('../results/logs_200_5.tsv',sep='\t', skiprows=[1,2])
df6 = pd.read_csv('../results/logs_200_6.tsv',sep='\t', skiprows=[1])
# only skip the first row for the next data frame


In [38]:
frames = [df1, df2, df3, df4, df5, df6]
logdf = pd.concat(frames)

In [39]:
logdf.shape

(996, 7)

In [40]:
logdf = logdf.sort_values(by=['Label','Date','Subject'])

In [41]:
logdf.to_csv('../results/1000_logs.tsv', sep='\t')

In [42]:
# get counts for unique rows of Label
by_label = logdf.groupby(['Label']).count()['FirstAuthor']

In [43]:
# get counts for number of unique labels by subject
by_label_subject = logdf.groupby(['Label', 'Subject']).count()['FirstAuthor']

In [44]:
# get counts for number of unique labels by time
by_label_time = logdf.groupby(['Label','Date']).count()['FirstAuthor']

#### actually trying the plots...

In [45]:
import matplotlib
%matplotlib inline
from IPython.display import HTML

In [46]:
html_pygal = u"""
    <!DOCTYPE html>
    <html>
        <head>
            <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
            <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/pygal-tooltips.js"></script>
        </head>
        <body><figure>{pygal_render}</figure></body>
    </html>
"""

In [47]:
#from pygal.style import Style
from pygal.style import CleanStyle
from pygal.style import DarkStyle


##### pie chart by gender

In [48]:
v = by_label.to_dict()

pie_chart = pygal.Pie(style=DarkStyle)
pie_chart.title = 'Breakdown of gender in first names'
for k,val in v.items():
    pie_chart.add(k,val)

In [49]:
HTML(html_pygal.format(pygal_render=pie_chart.render(is_unicode=True)))

In [50]:
#pie_chart.render_to_png('../results/figures/chart.png')
pie_chart.render_to_file('../results/figures/label_count.svg',isunicode=True)

##### bar chart by subject and label

In [51]:
subj_label = by_label_subject.to_dict()

subjects = pd.unique(logdf['Subject'])
labels = pd.unique(logdf['Label'])

In [52]:
plot_dict = {}
for i in labels:
    counter = []
    for j in subjects:
         counter.append(logdf[(logdf['Subject']==j) & (logdf['Label']==i)].count()['FirstAuthor'])
    plot_dict[i] = counter    

In [53]:
line_chart = pygal.Bar(x_label_rotation=90, style=DarkStyle)
line_chart.title = 'Gender Breakdown by Subject'
line_chart.x_labels = subjects

for k, v in plot_dict.items():
    line_chart.add(k, v)


In [54]:
HTML(html_pygal.format(pygal_render=line_chart.render(is_unicode=True)))

In [55]:
line_chart.render_to_file('../results/figures/label_subject.svg',isunicode=True)

#### papers by category

In [56]:
by_subject = logdf.groupby(['Subject']).count()['FirstAuthor']

In [57]:
paper_count = by_subject.to_dict()


In [58]:
line_chart = pygal.HorizontalBar(x_label_rotation=90, style=DarkStyle)
line_chart.title = 'Paper count by Subject'
#line_chart.x_labels = list(paper_count.keys())

for k, v in paper_count.items():
    line_chart.add(k, v)

In [59]:
HTML(html_pygal.format(pygal_render=line_chart.render(is_unicode=True)))

In [60]:
line_chart.render_to_file('../results/figures/count_subject.svg',isunicode=True)

#### by time and label

In [61]:
#logdf = logdf.sort_values(by=['Date'])
times = pd.unique(logdf['Date'])
labels = pd.unique(logdf['Label'])

In [62]:
plot_dict = {}
for i in labels:
    counter = []
    for j in times:
         counter.append(logdf[(logdf['Date']==j) & (logdf['Label']==i)].count()['FirstAuthor'])
    plot_dict[i] = counter  

In [63]:
#chart.add('First', [{'value': 2, 'label': 'This is the first'}])
line_chart = pygal.Bar(x_label_rotation=90, style=DarkStyle)
line_chart.title = 'Gender Breakdown by Timestamp'
line_chart.x_labels = times

for k, v in plot_dict.items():
    line_chart.add(k, v)

In [64]:
HTML(html_pygal.format(pygal_render=line_chart.render(is_unicode=True)))

In [65]:
line_chart.render_to_file('../results/figures/date_label.svg',isunicode=True)